#Instalando o PySpark no Google Colab


Instalar o PySpark não é um processo direto como de praxe em Python. Não basta usar um pip install apenas. Na verdade, antes de tudo é necessário instalar dependências como o Java 8, Apache Spark 2.3.2 junto com o Hadoop 2.7.

In [ ]:
# instalar as dependências
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

A próxima etapa é configurar as variáveis de ambiente, pois isso habilita o ambiente do Colab a identificar corretamente onde as dependências estão rodando.

Para conseguir “manipular” o terminal e interagir como ele, você pode usar a biblioteca os.

In [ ]:
# configurar as variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

# tornar o pyspark "importável"
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')

Com tudo pronto, vamos rodar uma sessão local para testar se a instalação funcionou corretamente.

In [ ]:
# iniciar uma sessão local e importar dados do Airbnb
from pyspark.sql import SparkSession
sc = SparkSession.builder.master('local[*]').getOrCreate()

# download do http para arquivo local
!wget --quiet --show-progress http://data.insideairbnb.com/brazil/rj/rio-de-janeiro/2019-07-15/visualisations/listings.csv

# carregar dados do Airbnb
df_spark = sc.read.csv("./listings.csv", inferSchema=True, header=True)

# ver algumas informações sobre os tipos de dados de cada coluna
df_spark.printSchema()

listings.csv        100%[===================>]   4.49M  9.03MB/s    in 0.5s    
root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: string (nullable = true)
 |-- minimum_nights: string (nullable = true)
 |-- number_of_reviews: string (nullable = true)
 |-- last_review: string (nullable = true)
 |-- reviews_per_month: string (nullable = true)
 |-- calculated_host_listings_count: double (nullable = true)
 |-- availability_365: integer (nullable = true)



Como resultado, você deve ver algo parecido com a imagem abaixo, o que significa que os nossos dados estão rodando corretamente junto ao PySpark.

#Contando palavras Estáticas

In [ ]:
#cria a seção a ser utilizada pelo spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('abc').getOrCreate()

In [ ]:
#lendo o arquivo de texto
read_file = spark\
.read\
.format("txt")\
.option("inferSchema", "true")\
.text("/content/arquivoTexto.txt")


In [ ]:
#print do esquema do arquivo lindo
read_file.printSchema()

root
 |-- value: string (nullable = true)



In [ ]:
#verifica se é um streaming
read_file.isStreaming

False

In [ ]:
#define as bibliotecas  a serem utilizadas
from pyspark.sql.functions import explode  #separa os elementos de uma linha em multiplas linhas
from pyspark.sql.functions import split    #divide as linhas

In [ ]:
#divide as linhas do arquivo
words = read_file.select(
    explode(
        split(read_file.value, " ")
    ).alias("word")
)

#cria o novo dataframe a ser responsável por contar a quantidade de palavras digitadas
wordCounts = words.groupBy("word").count()

In [ ]:
#realiza o "plot" das palavras
wordCounts.show()

+----------+-----+
|      word|count|
+----------+-----+
|    minhas|    1|
|   esqueci|    1|
|      vida|    1|
|esquecerei|    1|
|   caminho|    6|
|       tão|    1|
|      meio|    6|
|    pedra.|    2|
|     pedra|    4|
|        me|    2|
|        de|    1|
|     tinha|    7|
|     Nunca|    2|
|       que|    1|
|        No|    1|
|   retinas|    1|
|        do|    6|
|       uma|    7|
|   pedrano|    1|
|        no|    4|
+----------+-----+
only showing top 20 rows

